In [1]:
from sdv.demo import load_tabular_demo
import pandas as pd
import numpy as np
from sdv.tabular import TVAE
from sdv.constraints import OneHotEncoding
import os
import time
import warnings
warnings.filterwarnings("ignore")

In [4]:
def generate(data1,original_nums):
    #函数接收参数num，用以抽取num条原始数据并生成相应模型和数据到  ./data/n 文件夹下
    #  original_num 为字符串类型   
    
    # 1.接收并获取数据
    num = original_nums
    data = data1.sample(n=int(num))
    
    if(not os.path.exists('./data/'+num)): # path是文件夹或者文件的相对路径或者绝对路径
        os.mkdir('./data/'+num)
    
    # data.shape
    
    
    #这里是各种onehot的constraints
    addr_type_constraint = OneHotEncoding(
        column_names=['ADDRTYPEAlley','ADDRTYPEBlock','ADDRTYPEIntersection']
    )

    collision_type_constraint = OneHotEncoding(
        column_names=['COLLISIONTYPEAngles', 'COLLISIONTYPEHead.On', 'COLLISIONTYPELeft.Turn','COLLISIONTYPEParked.Car', 'COLLISIONTYPERear.Ended', 'COLLISIONTYPERight.Turn', 'COLLISIONTYPESideswipe']
    )

    month_constraint = OneHotEncoding(
        column_names=['Month1', 'Month10', 'Month11', 'Month12', 'Month2', 'Month3', 'Month4', 'Month5','Month6', 'Month7', 'Month8', 'Month9']
    )

    time_in_day_constraint = OneHotEncoding(
        column_names=['Timeevening_peak','Timemorning_peak', 'Timenon_peak']
    )

    # underinf_type_constraint = OneHotEncoding(  # 不知道什么意思
    #     column_names=['UNDERINFLN','UNDERINFLY']
    # )

    weather_condition_constraint = OneHotEncoding(
        column_names=['WEATHERBlowing.Sand.Dirt', 'WEATHERBlowing.Snow',
           'WEATHERClear', 'WEATHERFog.Smog.Smoke', 'WEATHEROvercast',
           'WEATHERPartly.Cloudy', 'WEATHERRaining', 'WEATHERSevere.Crosswind',
           'WEATHERSleet.Hail.Freezing.Rain', 'WEATHERSnowing']
    )

    road_condition_constraint = OneHotEncoding(
        column_names=['ROADCONDDry',
           'ROADCONDIce', 'ROADCONDOil', 'ROADCONDSand.Mud.Dirt',
           'ROADCONDSnow.Slush', 'ROADCONDStanding.Water', 'ROADCONDWet']
    )

    light_condition_constraint = OneHotEncoding(
        column_names=['LIGHTCONDDark...No.Street.Lights',
           'LIGHTCONDDark...Street.Lights.Off', 'LIGHTCONDDark...Street.Lights.On',
           'LIGHTCONDDawn', 'LIGHTCONDDaylight', 'LIGHTCONDDusk']
    )

    severity_condition_constraint = OneHotEncoding(
        column_names=['Severity_Fatality Collision',
           'Severity_Injury Collision', 'Severity_Property Damage Only Collision',
           'Severity_Serious Injury Collision']
    )

    constraints = [
        month_constraint,
        addr_type_constraint,
        collision_type_constraint,

        time_in_day_constraint,
    #     underinf_type_constraint,
        weather_condition_constraint,
        road_condition_constraint,
        light_condition_constraint,
        severity_condition_constraint
    ]
    model = TVAE(constraints=constraints, enforce_min_max_values=False)
    # t1 = time.time()
    model.fit(data)
    # t2 = time.time()
    # print('cost time = ',t2-t1)
    model.save('./data/'+num+'/'+num+'_model.pkl')

    load_model = TVAE.load('./data/'+num+'/'+num+'_model.pkl')
    
    data.to_csv('./data/'+num+'/'+num+'_real_data.csv')
    generate_data_5000 = load_model.sample(num_rows=5000)
    print(generate_data_5000.shape)
    generate_data_5000.to_csv('./data/'+num+'/'+num+'_to_5000.csv')
    generate_data_10000 = load_model.sample(num_rows=10000)
    print(generate_data_10000.shape)
    generate_data_5000.to_csv('./data/'+num+'/'+num+'_to_10000.csv')
    generate_data_20000 = load_model.sample(num_rows=20000)
    print(generate_data_20000.shape)
    generate_data_5000.to_csv('./data/'+num+'/'+num+'_to_20000.csv')

In [5]:
# data = load_tabular_demo('student_placements')
# data1 = pd.read_excel('Seattle_vv_21.xlsx')
# a = pd.read_excel('Processed_Seattle_vv_21_modifiedbyFDR.xlsx',index_col=0)
# a.to_csv('processed_seattle_vv_21.csv',encoding='utf-8')

data1 = pd.read_csv('processed_seattle_vv_21.csv')
data1 = data1.drop(["UNDERINFLY"],axis=1)
data1 = data1.drop(['UNDERINFLN'],axis=1)
data1 = pd.get_dummies(data1,columns=['Severity'],dtype=int)
# data1.dtypes
# data1.shape

for i in range(6,11):
    n = str(i)+'00'
    generate(data1,n)

Sampling rows: 100%|████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 10798.96it/s]


(5000, 52)


Sampling rows: 100%|██████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 16025.14it/s]


(10000, 52)


Sampling rows: 100%|██████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 20432.94it/s]


...

(5000, 52)


Sampling rows: 100%|██████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17945.93it/s]


(10000, 52)


Sampling rows: 100%|██████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 22461.13it/s]


(20000, 52)
